In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def parse_pssm(fname):
    f = open(fname)
    # the 4th line should be the start of the PSSM data
    f.readline()
    f.readline()
    f.readline()
    seq = []
    lprob = np.zeros([0,20])
    prob = np.zeros([0,20])
    extra = np.zeros([0,2])
    line = f.readline()
    while len(line.strip())>0:
        lineinfo = line.split()
        seq.append(lineinfo[1])
        lprobs_ = [float(lineinfo[i]) for i in range(2,22)]
        lprob = np.concatenate((lprob,np.matrix(lprobs_)),axis=0)
        probs_ = [float(lineinfo[i])/100 for i in range(22,42)]
        prob = np.concatenate((prob,np.matrix(probs_)),axis=0)
        extras_ = [float(lineinfo[i]) for i in range(42,44)]
        extra = np.concatenate((extra,np.matrix(extras_)),axis=0)
        line = f.readline()

    return (''.join(seq),prob,lprob,extra)

## PSSM Profile Extraction

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/HDA/Benchmark_BinaryML.csv')

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
pssm_matrices = []

for i in df['PDBid']:
    pssm_mat = parse_pssm(f'/content/drive/MyDrive/HDA/Benchmark_PSSM/{i}.txt')
    pssm_arr = np.array(pssm_mat[1])
    pssm_matrices.append(pssm_arr)

# Add the list of pssm_arr as a new column 'pssm_matrix' in the DataFrame
df['pssm_matrix'] = pssm_matrices

## Skipxgram Extraction

In [ ]:
import numpy as np

def calculate_sxgbg_features(evolutionary_profile, X):

    L, _ = evolutionary_profile.shape
    sxgbg_matrix = np.zeros((20, 20))

    for i in range(20):
        for j in range(20):
            sxgbg_value = 0.0

            for l in range(1,L - X):
                sxgbg_value += evolutionary_profile[l-1, i] * evolutionary_profile[l + X , j]

            sxgbg_matrix[i, j] = sxgbg_value

    return sxgbg_matrix

### S0G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s0g_matrix = []

for i in df['pssm_matrix']:
    s0g_mat = calculate_sxgbg_features(i, 0)
    s0g_arr = np.array(s0g_mat)
    s0g_matrix.append(s0g_arr)

df['s0g_matrix'] = s0g_matrix

### S1G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s1g_matrix = []

for i in df['pssm_matrix']:
    s1g_mat = calculate_sxgbg_features(i, 1)
    s1g_arr = np.array(s1g_mat)
    s1g_matrix.append(s1g_arr)

df['s1g_matrix'] = s1g_matrix

### S2G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s2g_matrix = []

for i in df['pssm_matrix']:
    s2g_mat = calculate_sxgbg_features(i, 2)
    s2g_arr = np.array(s2g_mat)
    s2g_matrix.append(s2g_arr)

df['s2g_matrix'] = s2g_matrix

### S3G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s3g_matrix = []

for i in df['pssm_matrix']:
    s3g_mat = calculate_sxgbg_features(i, 3)
    s3g_arr = np.array(s3g_mat)
    s3g_matrix.append(s3g_arr)

df['s3g_matrix'] = s3g_matrix

### S4G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s4g_matrix = []

for i in df['pssm_matrix']:
    s4g_mat = calculate_sxgbg_features(i, 4)
    s4g_arr = np.array(s4g_mat)
    s4g_matrix.append(s4g_arr)

df['s4g_matrix'] = s4g_matrix

### S5G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s5g_matrix = []

for i in df['pssm_matrix']:
    s5g_mat = calculate_sxgbg_features(i, 5)
    s5g_arr = np.array(s5g_mat)
    s5g_matrix.append(s5g_arr)

df['s5g_matrix'] = s5g_matrix

### S6G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s6g_matrix = []

for i in df['pssm_matrix']:
    s6g_mat = calculate_sxgbg_features(i, 6)
    s6g_arr = np.array(s6g_mat)
    s6g_matrix.append(s6g_arr)

df['s6g_matrix'] = s6g_matrix

## Evaluation Metrics

In [ ]:
def calculate_accuracy(y_true, y_pred):
    # Ensure inputs are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array((y_pred>0.5).astype(int))

    # Initialize accuracy
    accuracy = 0

    # Calculate accuracy for each instance
    for i in range(len(y_true)):
        # Calculate intersection and union
        intersection = np.sum(np.logical_and(y_true[i], y_pred[i]))
        union = np.sum(np.logical_or(y_true[i], y_pred[i]))

        # Add to total accuracy
        accuracy += intersection / union

    # Calculate average accuracy
    accuracy /= len(y_true)

    return accuracy

In [ ]:
import numpy as np

def norm_accuracy(y_true, y_pred):
    # Ensure inputs are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array((y_pred>0.5).astype(int))

    acc = []
    # Loop over each instance
    for i in range(len(y_true)):
        # Calculate the number of correct predictions for this instance
        print(y_true[i], y_pred[i])
        correct_predictions = np.sum(y_true[i] == y_pred[i])
        print(correct_predictions/5)
        acc.append(correct_predictions/5)

    # Calculate accuracy
    accuracy = sum(acc) / len(y_true)

    return accuracy

## S0G Matrix Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

# Define the model
model_s0g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s0g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s0g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s0g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s0g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.5073 - accuracy: 0.4567 - val_loss: 0.6545 - val_accuracy: 0.5948 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4401 - accuracy: 0.5476 - val_loss: 0.3579 - val_accuracy: 0.6121 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3695 - accuracy: 0.5693 - val_loss: 0.4243 - val_accuracy: 0.5431 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3205 - accuracy: 0.6147 - val_loss: 0.3566 - val_accuracy: 0.5690 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3092 - accuracy: 0.7013 - val_loss: 0.3210 - val_accuracy: 0.6293 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2831 - accuracy: 0.6883 - val_loss: 0.3047 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 1s 6ms/step - loss: 0.1268 - accuracy: 0.8683 - val_loss: 0.1680 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1230 - accuracy: 0.8877 - val_loss: 0.2362 - val_accuracy: 0.7652 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 4ms/step - loss: 0.0911 - accuracy: 0.9179 - val_loss: 0.2360 - val_accuracy: 0.7652 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1196 - accuracy: 0.8920 - val_loss: 0.1216 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0890 - accuracy: 0.9266 - val_loss: 0.1062 - val_accuracy: 0.9043 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0762 - accuracy: 0.9244 - val_loss: 0.1893 - val_accuracy: 0.8174 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

## S1G Matrix Model

In [ ]:
# Define the model
model_s1g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s1g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s1g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s1g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s1g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 1s 6ms/step - loss: 0.4940 - accuracy: 0.4589 - val_loss: 0.4249 - val_accuracy: 0.5259 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4201 - accuracy: 0.5584 - val_loss: 0.3805 - val_accuracy: 0.5345 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.3753 - accuracy: 0.5996 - val_loss: 0.3243 - val_accuracy: 0.6466 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.3378 - accuracy: 0.6234 - val_loss: 0.3902 - val_accuracy: 0.5603 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.3276 - accuracy: 0.6320 - val_loss: 0.2883 - val_accuracy: 0.6724 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2996 - accuracy: 0.6537 - val_loss: 0.2933 - val_accuracy: 0.7069 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2694 - accuracy: 0.7100 - val_loss: 0.2188 - val_accuracy: 0.7328 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2501 - accuracy: 0.7316 - val_loss: 0.2462 - val_accuracy: 0.6724 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2444 - accuracy: 0.7143 - val_loss: 0.2766 - val_accuracy: 0.7155 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2398 - accuracy: 0.7208 - val_loss: 0.2825 - val_accuracy: 0.6983 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2293 - accuracy: 0.7165 - val_loss: 0.2412 - val_accuracy: 0.7155 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2321 - accuracy: 0.7251 - val_loss: 0.2153 - val_accuracy: 0.7586 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 7ms/step - loss: 0.0963 - accuracy: 0.9201 - val_loss: 0.1122 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0798 - accuracy: 0.9287 - val_loss: 0.1559 - val_accuracy: 0.8870 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 7ms/step - loss: 0.1081 - accuracy: 0.9309 - val_loss: 0.1662 - val_accuracy: 0.8609 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0663 - accuracy: 0.9438 - val_loss: 0.1027 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.1018 - accuracy: 0.9201 - val_loss: 0.1431 - val_accuracy: 0.8348 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0690 - accuracy: 0.9395 - val_loss: 0.1027 - val_accuracy: 0.8957 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

## S2G Matrix Model

In [ ]:
# Define the model
model_s2g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s2g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s2g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s2g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s2g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.6369 - accuracy: 0.3160 - val_loss: 0.5012 - val_accuracy: 0.4741 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4529 - accuracy: 0.5000 - val_loss: 0.4588 - val_accuracy: 0.3362 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3894 - accuracy: 0.5476 - val_loss: 0.3642 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3695 - accuracy: 0.5909 - val_loss: 0.3635 - val_accuracy: 0.5345 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3561 - accuracy: 0.6190 - val_loss: 0.3527 - val_accuracy: 0.5517 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3262 - accuracy: 0.6602 - val_loss: 0.3432 - val_accuracy: 0.6293 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1964 - accuracy: 0.8355 - val_loss: 0.1396 - val_accuracy: 0.8534 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1548 - accuracy: 0.8658 - val_loss: 0.0934 - val_accuracy: 0.9397 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1522 - accuracy: 0.8636 - val_loss: 0.1038 - val_accuracy: 0.9138 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1223 - accuracy: 0.8788 - val_loss: 0.1095 - val_accuracy: 0.9052 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.1291 - accuracy: 0.8939 - val_loss: 0.1181 - val_accuracy: 0.8707 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 7ms/step - loss: 0.1182 - accuracy: 0.8788 - val_loss: 0.1511 - val_accuracy: 0.8621 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 6ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0837 - accuracy: 0.9244 - val_loss: 0.0929 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0929 - accuracy: 0.9071 - val_loss: 0.1201 - val_accuracy: 0.8696 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 6ms/step - loss: 0.0647 - accuracy: 0.9417 - val_loss: 0.0815 - val_accuracy: 0.9043 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0774 - accuracy: 0.9266 - val_loss: 0.1179 - val_accuracy: 0.8957 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0590 - accuracy: 0.9460 - val_loss: 0.1742 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0685 - accuracy: 0.9352 - val_loss: 0.1086 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 6ms/step - lo

## S3G Matrix Model

In [ ]:
# Define the model
model_s3g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s3g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s3g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s3g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s3g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.5024 - accuracy: 0.4697 - val_loss: 0.4539 - val_accuracy: 0.4569 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4084 - accuracy: 0.5476 - val_loss: 0.3575 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3707 - accuracy: 0.5866 - val_loss: 0.3221 - val_accuracy: 0.6983 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3436 - accuracy: 0.6299 - val_loss: 0.3555 - val_accuracy: 0.5948 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3509 - accuracy: 0.6190 - val_loss: 0.3162 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2997 - accuracy: 0.6558 - val_loss: 0.2852 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 10ms/step - loss: 0.1779 - accuracy: 0.8272 - val_loss: 0.2254 - val_accuracy: 0.7739 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1501 - accuracy: 0.8618 - val_loss: 0.1591 - val_accuracy: 0.8261 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1535 - accuracy: 0.8380 - val_loss: 0.1975 - val_accuracy: 0.7913 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1275 - accuracy: 0.8769 - val_loss: 0.2359 - val_accuracy: 0.7478 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1252 - accuracy: 0.8812 - val_loss: 0.2062 - val_accuracy: 0.8261 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0917 - accuracy: 0.9222 - val_loss: 0.1759 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - l

## S4G Matrix Model

In [ ]:
# Define the model
model_s4g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s4g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s4g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s4g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s4g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.5099 - accuracy: 0.4026 - val_loss: 0.4375 - val_accuracy: 0.4741 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4249 - accuracy: 0.5152 - val_loss: 0.3908 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3571 - accuracy: 0.6147 - val_loss: 0.3912 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 6ms/step - loss: 0.3618 - accuracy: 0.6190 - val_loss: 0.4396 - val_accuracy: 0.4914 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.3243 - accuracy: 0.6494 - val_loss: 0.3280 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 7ms/step - loss: 0.3007 - accuracy: 0.6645 - val_loss: 0.3083 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 3s 8ms/step - loss: 0.1135 - accuracy: 0.8985 - val_loss: 0.2073 - val_accuracy: 0.8261 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1219 - accuracy: 0.8920 - val_loss: 0.1042 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1032 - accuracy: 0.8920 - val_loss: 0.4837 - val_accuracy: 0.6435 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1113 - accuracy: 0.8790 - val_loss: 0.1714 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0836 - accuracy: 0.9266 - val_loss: 0.3154 - val_accuracy: 0.7043 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0808 - accuracy: 0.9374 - val_loss: 0.1358 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

## S5G Matrix Model

In [ ]:
# Define the model
model_s5g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s5g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s5g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s5g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s5g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.5330 - accuracy: 0.4048 - val_loss: 0.4249 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4512 - accuracy: 0.5108 - val_loss: 0.4253 - val_accuracy: 0.5172 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3807 - accuracy: 0.5714 - val_loss: 0.3762 - val_accuracy: 0.5862 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3640 - accuracy: 0.6234 - val_loss: 0.3367 - val_accuracy: 0.6034 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3440 - accuracy: 0.6255 - val_loss: 0.3120 - val_accuracy: 0.6552 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3095 - accuracy: 0.6732 - val_loss: 0.3203 - val_accuracy: 0.6293 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


116/116 [==============================] - 1s 6ms/step - loss: 0.1033 - accuracy: 0.9028 - val_loss: 0.0989 - val_accuracy: 0.9043 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1055 - accuracy: 0.8942 - val_loss: 0.1190 - val_accuracy: 0.8957 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0883 - accuracy: 0.8985 - val_loss: 0.1633 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 7ms/step - loss: 0.1200 - accuracy: 0.8877 - val_loss: 0.2198 - val_accuracy: 0.7913 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0753 - accuracy: 0.9222 - val_loss: 0.1559 - val_accuracy: 0.8348 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0812 - accuracy: 0.9244 - val_loss: 0.1509 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0921 

## S6G Matrix Model

In [ ]:
# Define the model
model_s6g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s6g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s6g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s6g_1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s6g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.5040 - accuracy: 0.3377 - val_loss: 0.4327 - val_accuracy: 0.4224 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.4570 - accuracy: 0.4870 - val_loss: 0.4375 - val_accuracy: 0.4569 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3852 - accuracy: 0.5649 - val_loss: 0.4653 - val_accuracy: 0.4224 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3848 - accuracy: 0.5498 - val_loss: 0.3896 - val_accuracy: 0.5948 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3355 - accuracy: 0.6342 - val_loss: 0.3717 - val_accuracy: 0.5431 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.3507 - accuracy: 0.5909 - val_loss: 0.3699 - val_accuracy: 0.5259 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 7ms/step - loss: 0.2695 - accuracy: 0.6970 - val_loss: 0.2704 - val_accuracy: 0.6552 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2812 - accuracy: 0.6948 - val_loss: 0.2698 - val_accuracy: 0.7414 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2588 - accuracy: 0.7208 - val_loss: 0.2992 - val_accuracy: 0.6552 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 8ms/step - loss: 0.2575 - accuracy: 0.7035 - val_loss: 0.2514 - val_accuracy: 0.7241 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2508 - accuracy: 0.7165 - val_loss: 0.3872 - val_accuracy: 0.5776 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 6ms/step - loss: 0.2692 - accuracy: 0.6926 - val_loss: 0.3248 - val_accuracy: 0.6466 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 5ms/step - loss: 0.2517 - accuracy: 0.7273 - val_loss: 0.2540 - val_accuracy: 0.6552 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2320 - accuracy: 0.7446 - val_loss: 0.2143 - val_accuracy: 0.7586 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2018 - accuracy: 0.7814 - val_loss: 0.2765 - val_accuracy: 0.6810 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1923 - accuracy: 0.7879 - val_loss: 0.2364 - val_accuracy: 0.7241 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2025 - accuracy: 0.7576 - val_loss: 0.2305 - val_accuracy: 0.7069 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1722 - accuracy: 0.7900 - val_loss: 0.2542 - val_accuracy: 0.7155 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.3020 - accuracy: 0.6566 - val_loss: 0.2114 - val_accuracy: 0.8261 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2779 - accuracy: 0.7084 - val_loss: 0.1626 - val_accuracy: 0.8609 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2341 - accuracy: 0.7387 - val_loss: 0.1583 - val_accuracy: 0.8435 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2130 - accuracy: 0.7646 - val_loss: 0.1229 - val_accuracy: 0.8522 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1897 - accuracy: 0.7624 - val_loss: 0.1420 - val_accuracy: 0.8087 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2089 - accuracy: 0.7495 - val_loss: 0.1185 - val_accuracy: 0.8435 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.2076 - accuracy: 0.7624 - val_loss: 0.2621 - val_accuracy: 0.6957 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 4ms/step - loss: 0.2027 - accuracy: 0.7797 - val_loss: 0.2507 - val_accuracy: 0.6957 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1955 - accuracy: 0.7754 - val_loss: 0.2555 - val_accuracy: 0.7130 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 4ms/step - loss: 0.1880 - accuracy: 0.7905 - val_loss: 0.4274 - val_accuracy: 0.5913 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.1951 - accuracy: 0.7970 - val_loss: 0.2709 - val_accuracy: 0.7217 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 7ms/step - loss: 0.1677 - accuracy: 0.8056 - val_loss: 0.2685 - val_accuracy: 0.6783 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Standardizing SkipGram

In [ ]:
# standardize the sxgbg_matrix
import numpy as np

def calculate_sxgbg_stan_features(evolutionary_profile, X):
    L, _ = evolutionary_profile.shape
    sxgbg_matrix = np.zeros((20, 20))

    for i in range(20):
        for j in range(20):
            sxgbg_value = 0.0

            for l in range(1, L - X):
                sxgbg_value += evolutionary_profile[l - 1, i] * evolutionary_profile[l + X, j]

            sxgbg_matrix[i, j] = sxgbg_value

    # Standardize the sxgbg_matrix (z-score normalization)
    mean = np.mean(sxgbg_matrix)
    std = np.std(sxgbg_matrix)

    if std != 0:
        sxgbg_matrix = (sxgbg_matrix - mean) / std
    else:
        sxgbg_matrix = np.zeros_like(sxgbg_matrix)  # Handle the case of zero standard deviation

    return sxgbg_matrix

## S0G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s0g_stan_matrix = []

for i in df['pssm_matrix']:
    s0g_mat = calculate_sxgbg_stan_features(i, 0)
    s0g_arr = np.array(s0g_mat)
    s0g_stan_matrix.append(s0g_arr)

df['s0g_stan_matrix'] = s0g_stan_matrix

## S1G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s1g_stan_matrix = []

for i in df['pssm_matrix']:
    s1g_mat = calculate_sxgbg_stan_features(i, 1)
    s1g_arr = np.array(s1g_mat)
    s1g_stan_matrix.append(s1g_arr)

df['s1g_stan_matrix'] = s1g_stan_matrix

## S2G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s2g_stan_matrix = []

for i in df['pssm_matrix']:
    s2g_mat = calculate_sxgbg_stan_features(i, 2)
    s2g_arr = np.array(s2g_mat)
    s2g_stan_matrix.append(s2g_arr)

df['s2g_stan_matrix'] = s2g_stan_matrix

## S3G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s3g_stan_matrix = []

for i in df['pssm_matrix']:
    s3g_mat = calculate_sxgbg_stan_features(i, 3)
    s3g_arr = np.array(s3g_mat)
    s3g_stan_matrix.append(s3g_arr)

df['s3g_stan_matrix'] = s3g_stan_matrix

## S4G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s4g_stan_matrix = []

for i in df['pssm_matrix']:
    s4g_mat = calculate_sxgbg_stan_features(i, 4)
    s4g_arr = np.array(s4g_mat)
    s4g_stan_matrix.append(s4g_arr)

df['s4g_stan_matrix'] = s4g_stan_matrix

## S5G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s5g_stan_matrix = []

for i in df['pssm_matrix']:
    s5g_mat = calculate_sxgbg_stan_features(i, 5)
    s5g_arr = np.array(s5g_mat)
    s5g_stan_matrix.append(s5g_arr)

df['s5g_stan_matrix'] = s5g_stan_matrix

## S6G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s6g_stan_matrix = []

for i in df['pssm_matrix']:
    s6g_mat = calculate_sxgbg_stan_features(i, 6)
    s6g_arr = np.array(s6g_mat)
    s6g_stan_matrix.append(s6g_arr)

df['s6g_stan_matrix'] = s6g_stan_matrix

## S0G Stan Matrix Model

In [ ]:
# Define the model
model_s0g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s0g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s0g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s0g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s0g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.4265 - accuracy: 0.5108 - val_loss: 0.3494 - val_accuracy: 0.5690 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2948 - accuracy: 0.6883 - val_loss: 0.3091 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2627 - accuracy: 0.6948 - val_loss: 0.2894 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1974 - accuracy: 0.8052 - val_loss: 0.3141 - val_accuracy: 0.6466 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1523 - accuracy: 0.8723 - val_loss: 0.3962 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1384 - accuracy: 0.8723 - val_loss: 0.2629 - val_accuracy: 0.6810 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 10ms/step - loss: 0.0571 - accuracy: 0.9525 - val_loss: 0.0335 - val_accuracy: 0.9652 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 8ms/step - loss: 0.0380 - accuracy: 0.9633 - val_loss: 0.0393 - val_accuracy: 0.9739 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0230 - accuracy: 0.9676 - val_loss: 0.0299 - val_accuracy: 0.9478 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 4ms/step - loss: 0.0222 - accuracy: 0.9698 - val_loss: 0.1999 - val_accuracy: 0.8609 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0485 - accuracy: 0.9590 - val_loss: 0.0943 - val_accuracy: 0.9478 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0069 - accuracy: 0.9849 - val_loss: 0.0642 - val_accuracy: 0.9652 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 4ms/step - l

## S1G Stan Matrix Model

In [ ]:
# Define the model
model_s1g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s1g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s1g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s1g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s1g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.4050 - accuracy: 0.5195 - val_loss: 0.3241 - val_accuracy: 0.6207 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.3132 - accuracy: 0.6299 - val_loss: 0.3036 - val_accuracy: 0.5690 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2378 - accuracy: 0.7619 - val_loss: 0.3609 - val_accuracy: 0.5603 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2124 - accuracy: 0.7684 - val_loss: 0.2726 - val_accuracy: 0.6552 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.1840 - accuracy: 0.8095 - val_loss: 0.2619 - val_accuracy: 0.6897 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 7ms/step - loss: 0.1276 - accuracy: 0.8766 - val_loss: 0.2849 - val_accuracy: 0.6897 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0492 - accuracy: 0.9568 - val_loss: 0.0624 - val_accuracy: 0.9217 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0402 - accuracy: 0.9568 - val_loss: 0.0932 - val_accuracy: 0.9391 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0462 - accuracy: 0.9568 - val_loss: 0.3036 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 6ms/step - loss: 0.0516 - accuracy: 0.9503 - val_loss: 0.0577 - val_accuracy: 0.9565 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0306 - accuracy: 0.9568 - val_loss: 0.0375 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 7ms/step - loss: 0.0284 - accuracy: 0.9611 - val_loss: 0.1028 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 7ms/step - lo

## S2G Stan Matrix Model

In [ ]:
# Define the model
model_s2g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s2g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s2g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s2g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s2g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 1s 6ms/step - loss: 0.4361 - accuracy: 0.4719 - val_loss: 0.4086 - val_accuracy: 0.4483 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 4ms/step - loss: 0.3138 - accuracy: 0.6450 - val_loss: 0.3183 - val_accuracy: 0.5862 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2322 - accuracy: 0.7619 - val_loss: 0.3138 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.2015 - accuracy: 0.7965 - val_loss: 0.2940 - val_accuracy: 0.6724 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1759 - accuracy: 0.8160 - val_loss: 0.2911 - val_accuracy: 0.7241 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1475 - accuracy: 0.8485 - val_loss: 0.2907 - val_accuracy: 0.7155 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 7ms/step - loss: 0.0652 - accuracy: 0.9503 - val_loss: 0.1010 - val_accuracy: 0.9043 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0572 - accuracy: 0.9417 - val_loss: 0.0779 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0415 - accuracy: 0.9568 - val_loss: 0.1857 - val_accuracy: 0.8609 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0505 - accuracy: 0.9546 - val_loss: 0.1575 - val_accuracy: 0.8609 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0384 - accuracy: 0.9546 - val_loss: 0.0770 - val_accuracy: 0.9217 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.0248 - accuracy: 0.9633 - val_loss: 0.0562 - val_accuracy: 0.9043 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

## S3G Stan Matrix Model

In [ ]:
# Define the model
model_s3g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s3g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s3g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s3g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s3g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 9ms/step - loss: 0.4161 - accuracy: 0.5022 - val_loss: 0.3048 - val_accuracy: 0.6552 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 7ms/step - loss: 0.3012 - accuracy: 0.6602 - val_loss: 0.2788 - val_accuracy: 0.6810 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 7ms/step - loss: 0.2438 - accuracy: 0.7511 - val_loss: 0.2561 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1990 - accuracy: 0.7965 - val_loss: 0.2908 - val_accuracy: 0.6810 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1610 - accuracy: 0.8268 - val_loss: 0.2959 - val_accuracy: 0.6897 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 0s 4ms/step - loss: 0.1471 - accuracy: 0.8485 - val_loss: 0.2644 - val_accuracy: 0.7672 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 0s 4ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 3s 9ms/step - loss: 0.0717 - accuracy: 0.9352 - val_loss: 0.1094 - val_accuracy: 0.9043 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0612 - accuracy: 0.9374 - val_loss: 0.1456 - val_accuracy: 0.8870 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0541 - accuracy: 0.9482 - val_loss: 0.0779 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0564 - accuracy: 0.9395 - val_loss: 0.1100 - val_accuracy: 0.8957 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0552 - accuracy: 0.9525 - val_loss: 0.0807 - val_accuracy: 0.9217 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 6ms/step - loss: 0.0320 - accuracy: 0.9698 - val_loss: 0.1652 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 6ms/step - lo

## S4G Stan Matrix Model

In [ ]:
# Define the model
model_s4g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s4g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s4g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s4g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s4g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 2s 9ms/step - loss: 0.3969 - accuracy: 0.5173 - val_loss: 0.2974 - val_accuracy: 0.6121 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 9ms/step - loss: 0.3212 - accuracy: 0.6342 - val_loss: 0.2923 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 9ms/step - loss: 0.2448 - accuracy: 0.7424 - val_loss: 0.3495 - val_accuracy: 0.6207 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 9ms/step - loss: 0.2184 - accuracy: 0.7965 - val_loss: 0.2841 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 7ms/step - loss: 0.1851 - accuracy: 0.7965 - val_loss: 0.2841 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1505 - accuracy: 0.8463 - val_loss: 0.3170 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 7ms/step - loss: 0.0675 - accuracy: 0.9352 - val_loss: 0.0530 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 6ms/step - loss: 0.0604 - accuracy: 0.9525 - val_loss: 0.0750 - val_accuracy: 0.9217 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0309 - accuracy: 0.9590 - val_loss: 0.0492 - val_accuracy: 0.9391 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0318 - accuracy: 0.9784 - val_loss: 0.1780 - val_accuracy: 0.8261 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 6ms/step - loss: 0.0396 - accuracy: 0.9676 - val_loss: 0.0478 - val_accuracy: 0.9478 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0314 - accuracy: 0.9611 - val_loss: 0.0805 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - lo

## S5G Stan Matrix Model

In [ ]:
# Define the model
model_s5g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s5g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s5g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s5g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s5g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 3s 11ms/step - loss: 0.4223 - accuracy: 0.4913 - val_loss: 0.3329 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2985 - accuracy: 0.6558 - val_loss: 0.3203 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 6ms/step - loss: 0.2468 - accuracy: 0.7359 - val_loss: 0.2647 - val_accuracy: 0.7672 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1994 - accuracy: 0.7900 - val_loss: 0.2588 - val_accuracy: 0.7759 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1745 - accuracy: 0.8247 - val_loss: 0.2599 - val_accuracy: 0.7759 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 6ms/step - loss: 0.1473 - accuracy: 0.8550 - val_loss: 0.4035 - val_accuracy: 0.6121 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 3s 12ms/step - loss: 0.2208 - accuracy: 0.7749 - val_loss: 0.1526 - val_accuracy: 0.8362 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 10ms/step - loss: 0.1772 - accuracy: 0.8290 - val_loss: 0.1463 - val_accuracy: 0.8793 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 9ms/step - loss: 0.1347 - accuracy: 0.8571 - val_loss: 0.1797 - val_accuracy: 0.8103 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1379 - accuracy: 0.8571 - val_loss: 0.1810 - val_accuracy: 0.8362 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1087 - accuracy: 0.9026 - val_loss: 0.1740 - val_accuracy: 0.8190 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0743 - accuracy: 0.9221 - val_loss: 0.1775 - val_accuracy: 0.8448 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 7ms/step - loss: 0.0954 - accuracy: 0.9050 - val_loss: 0.0790 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0654 - accuracy: 0.9395 - val_loss: 0.1070 - val_accuracy: 0.8783 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0497 - accuracy: 0.9438 - val_loss: 0.1094 - val_accuracy: 0.9217 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0495 - accuracy: 0.9568 - val_loss: 0.1266 - val_accuracy: 0.8609 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0480 - accuracy: 0.9503 - val_loss: 0.1827 - val_accuracy: 0.8087 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0506 - accuracy: 0.9438 - val_loss: 0.1390 - val_accuracy: 0.8696 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - lo

## S6G Stan Matrix Model

In [ ]:
# Define the model
model_s6g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s6g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s6g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=10, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 4
    model_s6g_stan.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s6g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/30
116/116 [==============================] - 3s 12ms/step - loss: 0.4178 - accuracy: 0.4784 - val_loss: 0.3459 - val_accuracy: 0.5948 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 6ms/step - loss: 0.3073 - accuracy: 0.6645 - val_loss: 0.2754 - val_accuracy: 0.6810 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2468 - accuracy: 0.7532 - val_loss: 0.2635 - val_accuracy: 0.7155 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.2007 - accuracy: 0.8009 - val_loss: 0.3368 - val_accuracy: 0.6379 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 5ms/step - loss: 0.1660 - accuracy: 0.8225 - val_loss: 0.2627 - val_accuracy: 0.7241 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 6ms/step - loss: 0.1452 - accuracy: 0.8680 - val_loss: 0.2454 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/30
116/116 [==============================] - 2s 6ms/step - loss: 0.0531 - accuracy: 0.9482 - val_loss: 0.0390 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 2/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0461 - accuracy: 0.9546 - val_loss: 0.0484 - val_accuracy: 0.9391 - lr: 0.0010
Epoch 3/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0367 - accuracy: 0.9741 - val_loss: 0.0567 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 4/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0222 - accuracy: 0.9654 - val_loss: 0.0659 - val_accuracy: 0.8957 - lr: 0.0010
Epoch 5/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0650 - accuracy: 0.9438 - val_loss: 0.0366 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 6/30
116/116 [==============================] - 1s 5ms/step - loss: 0.0150 - accuracy: 0.9741 - val_loss: 0.0327 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 7/30
116/116 [==============================] - 1s 5ms/step - lo